In [1]:
with open(file='/kaggle/input/melville-moby-dick/melville-moby_dick.txt', mode='r', encoding='utf-8') as input_fp:
    lines = input_fp.readlines()
print(len(lines))

22423


In [2]:
starts = [index for index, line in enumerate(lines) if line.startswith('CHAPTER')]
documents = []
for index, start in enumerate(starts):
    if index != len(starts)-1:
        end = starts[index+1]
    else:
        end = -1
    document = ' '.join(lines[start:end])
    documents.append(document)
for document in documents:
    print(len(document))

12534
8143
32809
9347
4302
4942
5563
5534
20197
8923
4043
5007
9936
4445
6757
31297
12978
7901
6773
5281
6354
9419
2238
9947
1671
7187
9733
8264
6737
1654
4602
30622
5942
12974
15051
16326
2873
2167
1510
9237
22428
21607
1799
12225
20568
6122
5501
23053
4797
5869
8884
4156
9582
46028
11183
7747
5699
5896
5478
8394
11401
3224
2904
16972
5618
3709
4293
6942
2586
5175
13508
9558
11936
9314
6989
5178
3903
9514
5515
5289
25036
6653
4648
4805
11742
10678
28307
7278
8382
6066
14455
5651
9475
7489
2892
10513
1396
5956
13736
15316
10167
9310
5333
8573
9275
5573
6370
8854
5268
12855
2567
5683
7066
3869
5290
3079
2626
5174
14387
1088
3546
321
7011
7149
6387
7735
4097
8073
3152
9839
2559
9047
20904
19119
27324
